In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
## print shape of dataset with row
train['train_or_test']='train'
test['train_or_test']='test'
dataset=train.append(test)
# let's make a function to create 3 variables from Age:
# 1-e filling NA with median, 2- random sampling or 3- zeroes

def impute_na(df, variable, median,mean):
    df[variable+'_NA'] = np.where(df[variable].isnull(), 1, 0)
    df[variable+'_median'] = df[variable].fillna(median)
    df[variable+'_mean'] = df[variable].fillna(mean)
    # random sampling
    df[variable+'_random'] = df[variable]
    # extract the random sample to fill the na
    random_sample = dataset[variable].dropna().sample(df[variable].isnull().sum(), random_state=0)
    # pandas needs to have the same index in order to merge datasets
    random_sample.index = df[df[variable].isnull()].index
    df.loc[df[variable].isnull(), variable+'_random'] = random_sample
    
    
median=dataset.Number_Weeks_Used.median()
mean=dataset.Number_Weeks_Used.mean()
impute_na(dataset, 'Number_Weeks_Used', median, mean)

dataset['Crop_Soil']=dataset['Crop_Type']+dataset['Soil_Type']

dataset['Doses_given']=dataset['Number_Doses_Week']*dataset['Number_Weeks_Used_random']
dataset['Doses_missed']=dataset['Number_Doses_Week']*dataset['Number_Weeks_Quit']

ohe1=pd.get_dummies(dataset[ 'Pesticide_Use_Category'],prefix='Pest_Cat_', drop_first=True)

ohe2 = pd.get_dummies(dataset[ 'Season'],prefix='Season_Cat_', drop_first=True)

final_ohe = pd.concat([dataset, ohe1, ohe2], axis=1, sort=False)

trn = final_ohe.loc[final_ohe['train_or_test']=='train', :]
tst = final_ohe.loc[final_ohe['train_or_test']=='test', :]

In [4]:
X_trn=trn[['Estimated_Insects_Count','Number_Doses_Week','Number_Weeks_Used_median', 'Pesticide_Use_Category',
       'Number_Weeks_Quit','Soil_Type','Crop_Type','Season']]
y=trn['Crop_Damage']
X_tst=tst[['Estimated_Insects_Count','Number_Doses_Week','Number_Weeks_Used_median', 'Pesticide_Use_Category',
       'Number_Weeks_Quit','Soil_Type','Crop_Type','Season']]

In [10]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_trn, y, test_size = 0.2, random_state = 0)


In [11]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout


# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'he_uniform',activation='relu',input_dim = 8))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'he_uniform',activation='relu'))
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'glorot_uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'Adamax', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
model_history=classifier.fit(X_train, y_train,validation_split=0.33, batch_size = 10, nb_epoch = 100)

# list all data in history

print(model_history.history.keys())
# summarize history for accuracy
plt.plot(model_history.history['acc'])
plt.plot(model_history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

<ipython-input-13-088f577e8541>:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=8, units=6, kernel_initializer="he_uniform")`
  classifier.add(Dense(output_dim = 6, init = 'he_uniform',activation='relu',input_dim = 8))
<ipython-input-13-088f577e8541>:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="he_uniform")`
  classifier.add(Dense(output_dim = 6, init = 'he_uniform',activation='relu'))
<ipython-input-13-088f577e8541>:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="glorot_uniform")`
  classifier.add(Dense(output_dim = 1, init = 'glorot_uniform', activation = 'sigmoid'))
<ipython-input-13-088f577e8541>:24: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  model_history=classifier.fit(X_train, y_train,validation_split=0.33, batch_size = 10, nb_epoch = 100)


Train on 47627 samples, validate on 23459 samples
Epoch 1/100
47627/47627 [==============================] - 7s 144us/step - loss: 0.4509 - accuracy: 0.8221 - val_loss: 0.4110 - val_accuracy: 0.8278
Epoch 2/100
47627/47627 [==============================] - 5s 101us/step - loss: 0.4032 - accuracy: 0.8254 - val_loss: 0.3943 - val_accuracy: 0.8248
Epoch 3/100
47627/47627 [==============================] - 5s 100us/step - loss: 0.3922 - accuracy: 0.8251 - val_loss: 0.3873 - val_accuracy: 0.8278
Epoch 4/100
47627/47627 [==============================] - 5s 100us/step - loss: 0.3866 - accuracy: 0.8265 - val_loss: 0.3831 - val_accuracy: 0.8280
Epoch 5/100
47627/47627 [==============================] - 5s 104us/step - loss: 0.3832 - accuracy: 0.8270 - val_loss: 0.3810 - val_accuracy: 0.8254
Epoch 6/100
47627/47627 [==============================] - 5s 95us/step - loss: 0.3806 - accuracy: 0.8279 - val_loss: 0.3776 - val_accuracy: 0.8295
Epoch 7/100
47627/47627 [==============================] 

47627/47627 [==============================] - 4s 85us/step - loss: 0.2271 - accuracy: 0.8276 - val_loss: 0.2699 - val_accuracy: 0.8276
Epoch 56/100
47627/47627 [==============================] - 4s 89us/step - loss: 0.2202 - accuracy: 0.8269 - val_loss: 0.2620 - val_accuracy: 0.8291
Epoch 57/100
47627/47627 [==============================] - 4s 93us/step - loss: 0.2131 - accuracy: 0.8288 - val_loss: 0.2616 - val_accuracy: 0.8202
Epoch 58/100
47627/47627 [==============================] - 5s 95us/step - loss: 0.2059 - accuracy: 0.8273 - val_loss: 0.2522 - val_accuracy: 0.8314
Epoch 59/100
47627/47627 [==============================] - 4s 87us/step - loss: 0.1979 - accuracy: 0.8268 - val_loss: 0.2488 - val_accuracy: 0.8333
Epoch 60/100
47627/47627 [==============================] - 4s 85us/step - loss: 0.1897 - accuracy: 0.8268 - val_loss: 0.2415 - val_accuracy: 0.8315
Epoch 61/100
47627/47627 [==============================] - 4s 88us/step - loss: 0.1823 - accuracy: 0.8269 - val_loss: 

KeyError: 'acc'